# `apps_long.csv`

In [7]:
library(dplyr)
library(readr)
library(tidyr)

cr_apps <- read_csv("../output/apps_long.csv", show_col_types = FALSE)

pl_ssns <- read_csv("../output/player_ssns.csv", show_col_types = FALSE)

gls <- read_csv("../output/scorers-long.csv", show_col_types = FALSE) %>%
    group_by(
        season,
        game_no,
        player_name
    ) %>%
    summarise(
        goals_scored = sum(goals_scored),
        .groups = "drop"
    )

res <- read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", show_col_types = FALSE)

In [8]:
library(stringr)

sb_game_ids <- read_csv("https://raw.githubusercontent.com/petebrown/update-player-stats/main/data/players_df.csv", show_col_types = FALSE) %>%
    select(
        sb_game_id,
        game_date        
    ) %>%
    rename(
        game_id = sb_game_id
    ) %>%
    mutate(
        game_id = str_remove(game_id, "tpg"),
        game_id = as.numeric(game_id)
    ) %>%
    unique() %>%
    arrange(
        game_date
    )

head(sb_game_ids, 3)

game_id,game_date
<dbl>,<date>
240309,1996-08-17
243200,1996-08-21
242383,1996-08-23


#### Pivot apps_long to wide format mimicking book

In [9]:
apps_long <- cr_apps %>%
    left_join(
        pl_ssns %>%
            select(
                season,
                disam_name,
                player_name
            ) %>% unique(),
        by = c("season" = "season", "player_name" = "disam_name")
    ) %>%
    mutate(
        player_name = player_name.y
    ) %>%
    select(-player_name.y) %>%
    left_join(
        gls,
        by = c("season" = "season", "game_no" = "game_no", "player_name" = "player_name")
    ) %>%
    left_join(
        res %>%
            select(
                season,
                ssn_game_no,
                generic_comp
            ),
        by = c("season" = "season", "game_no" = "ssn_game_no")
    ) %>%
    group_by(
        season,
        player_name,
        generic_comp,
        role
    ) %>%
    summarise(
        count = n(),
        .groups = "drop"
    )

# Print example for one season and player
apps_long %>%
    filter(
        season == "1989/90",
        player_name == "Johnny Morrissey"
    )    

season,player_name,generic_comp,role,count
<chr>,<chr>,<chr>,<chr>,<int>
1989/90,Johnny Morrissey,Associate Members' Cup,starter,4
1989/90,Johnny Morrissey,Associate Members' Cup,sub,1
1989/90,Johnny Morrissey,FA Cup,starter,1
1989/90,Johnny Morrissey,Football League,starter,24
1989/90,Johnny Morrissey,Football League,sub,3
1989/90,Johnny Morrissey,League Cup,starter,6


In [10]:
gls_long <- cr_apps %>%
    left_join(
        pl_ssns %>%
            select(
                season,
                disam_name,
                player_name
            ) %>% unique(),
        by = c("season" = "season", "player_name" = "disam_name")
    ) %>%
    mutate(
        player_name = player_name.y
    ) %>%
    select(-player_name.y) %>%
    left_join(
        gls %>%
            select(
                season,
                game_no,
                player_name,
                goals_scored
            ),
        by = c("season", "game_no", "player_name")
    ) %>%
    select(
        season,
        game_no,
        player_name,,
        goals_scored
    ) %>%
    left_join(
        res %>%
            select(
                season,
                ssn_game_no,
                generic_comp
            ),
        by = c("season" = "season", "game_no" = "ssn_game_no")
    ) %>%
    mutate(
        goals_scored = ifelse(is.na(goals_scored), 0, goals_scored)
    ) %>%
    group_by(
        season,
        player_name,
        generic_comp
    ) %>%
    summarise(
        count = sum(goals_scored),
        .groups = "drop"
    ) %>%
    mutate(
        role = "goals"
    )

# Print example for one season and player
gls_long %>%
    filter(
        season == "1989/90",
        player_name == "Johnny Morrissey"
    )

season,player_name,generic_comp,count,role
<chr>,<chr>,<chr>,<dbl>,<chr>
1989/90,Johnny Morrissey,Associate Members' Cup,1,goals
1989/90,Johnny Morrissey,FA Cup,0,goals
1989/90,Johnny Morrissey,Football League,5,goals
1989/90,Johnny Morrissey,League Cup,0,goals


In [11]:
# Change options so all columns show for season
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [12]:
rbind(apps_long, gls_long) %>%
    mutate(
        generic_comp = ifelse(generic_comp %in% c("Football League", "Non-League"), "League", generic_comp)
    ) %>%
    filter(
        season == "1987/88"
    ) %>%
    mutate(
        generic_comp = factor(
            generic_comp,
            levels = c("League", "FA Cup", "League Cup", "Associate Members' Cup", "Anglo-Italian Cup", "Full Members' Cup", "FA Trophy")
        ),
        role = factor(
            role,
            levels = c("starter", "sub", "goals")
        )
    ) %>%
    left_join(
        pl_ssns %>% select(season, player_name, disam_name) %>% unique(),
        by = c("season" = "season", "player_name" = "player_name")
    ) %>%
    arrange(
        season,
        disam_name
    ) %>%
    pivot_wider(
        id_cols = c(season, generic_comp, role),
        names_from = disam_name,
        values_from = count,
        values_fill = 0
    ) %>%
    arrange(
        season,
        generic_comp,
        role
    )

season,generic_comp,role,Aspinall,Bishop,Chamberlain,Craven,Garnett,Gorton,Hall,Harvey,Higgins,Hughes,Malkin,Martindale,McCarrick,McKenna,Moore,Morrissey,Muir,Mungall,Murray,Nixon,O'Rourke,Steel,Thorpe,Vickers,Williams G
<chr>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1987/88,League,starter,11,2,15,6,0,1,0,33,31,19,3,34,40,13,30,35,43,45,11,8,22,29,13,46,16
1987/88,League,sub,1,3,0,7,1,0,1,0,2,1,2,0,0,1,0,4,0,0,9,0,0,0,1,0,8
1987/88,League,goals,1,1,0,0,0,0,0,3,1,0,0,4,5,3,0,4,27,0,1,0,0,7,0,1,0
1987/88,FA Cup,starter,0,0,0,0,0,0,0,2,2,0,0,2,2,0,2,0,2,2,2,0,2,0,0,2,2
1987/88,FA Cup,goals,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0
1987/88,League Cup,starter,0,0,0,2,0,0,0,0,0,2,0,1,2,1,0,2,2,2,0,0,2,0,2,2,2
1987/88,League Cup,sub,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1987/88,League Cup,goals,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1987/88,Associate Members' Cup,starter,1,0,0,0,0,0,0,2,0,2,1,1,1,0,1,2,2,2,0,0,2,0,1,2,2
